In [1]:
import os
import sys

notebook_dir = os.getcwd()
project_root_path = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root_path)

from src import ModelXtoC
from scripts.run_preprocessing import preprocessing_main
from src.utils import *
from config import PROJECT_ROOT
from src.training import run_epoch_x_to_c

from src.utils import find_class_imbalance
from config import N_TRIMMED_CONCEPTS, N_CLASSES
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
torch.mps.empty_cache()

In [4]:
concept_labels, train_loader, test_loader = preprocessing_main(class_concepts=False, verbose=True)

Found 11788 images.
Processing in 369 batches of size 32 (for progress reporting)...


Processing batches: 100%|█████████████████████| 369/369 [01:00<00:00,  6.07it/s]



Finished processing.
Successfully transformed: 11788 images.
Found 11788 unique images.
Found 312 unique concepts.
Generated concept matrix with shape: (11788, 312)
Found 200 classes.
Found labels for 11788 images.
Generated one-hot matrix with shape: (11788, 200)
Split complete: 5994 train images, 5794 test images.
Dataset initialized with 5994 pre-sorted items.
Dataset initialized with 5794 pre-sorted items.


**Find device to run model on (CPU or GPU).**

In [5]:
device = torch.device("cuda" if torch.cuda.is_available()
                    else "mps" if torch.backends.mps.is_available()
                    else "cpu")
print(f"Using device: {device}")

Using device: mps


### Loss


In [6]:
use_weighted_loss = True # Set to False for simple unweighted loss

if use_weighted_loss:
    concept_weights = find_class_imbalance(concept_labels)
    attr_criterion = [nn.BCEWithLogitsLoss(weight=torch.tensor([ratio], device=device, dtype=torch.float))
                    for ratio in concept_weights]
else:
    attr_criterion = [nn.BCEWithLogitsLoss() for _ in range(N_TRIMMED_CONCEPTS)]

# Load instance-based model

In [ ]:
model = torch.load(os.path.join(PROJECT_ROOT, 'models', 'instance_level_model.pth'), map_location=device, weights_only=False)

In [10]:
if test_loader:
    with torch.no_grad():
        shuffled_labels = []

        # Iterate through all batches
        for batch in test_loader:
            _, _, image_labels, _ = batch
            # Append batch labels to our list
            shuffled_labels.append(image_labels)

        # Concatenate all batches into a single tensor
        shuffled_labels = torch.cat(shuffled_labels, dim=0)

        test_loss, test_acc, outputs = run_epoch_x_to_c(model, test_loader, attr_criterion, optimizer=None, n_concepts=N_TRIMMED_CONCEPTS, device=device, return_outputs='sigmoid', verbose=True)

# print(f"Shuffled labels shape: {shuffled_labels.shape}")
np.save(os.path.join(PROJECT_ROOT, 'output', 'Y_test_instance.npy'), shuffled_labels)
print(f'Best Model Summary   | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')

Best Model Summary   | Loss: 22.0923 | Acc: 80.318


In [9]:
def get_outputs_as_array(outputs, n_concepts):
    # Initialize an empty list to collect batches
    batch_results = []

    for i in range(len(outputs)):
        batch_size = outputs[i].shape[0]

        # Create a batch matrix with N_CONCEPTS number of columns
        batch_matrix = np.zeros((batch_size, n_concepts))

        num_examples = len(outputs[i])

        # Fill the matrix only with the available concept values
        concepts_to_process = min(num_examples, n_concepts)

        for instance_idx in range(concepts_to_process):
            # Extract, convert, and flatten data for the current concept
            instance_data = outputs[i][instance_idx].detach().cpu().numpy().flatten()
            batch_matrix[instance_idx, :] = instance_data
        # Add this consistently shaped batch matrix to our collection
        batch_results.append(batch_matrix)

    return np.vstack(batch_results)

In [12]:
output_array = get_outputs_as_array(outputs, N_TRIMMED_CONCEPTS)
print(f"Final shape: {output_array.shape}")

np.save(os.path.join(PROJECT_ROOT, 'output', 'C_hat_sigmoid_test_instance.npy'), output_array)

Final shape: (5794, 112)


In [14]:
output_array[0]

array([0.02501685, 0.77495462, 0.0687966 , 0.09519394, 0.14613137,
       0.48839876, 0.04856971, 0.16711   , 0.40154633, 0.09964222,
       0.14682958, 0.45817006, 0.00804877, 0.09553508, 0.44051   ,
       0.09777722, 0.02271112, 0.19730091, 0.03049137, 0.0149068 ,
       0.80746418, 0.08676503, 0.70120353, 0.04629159, 0.09214313,
       0.06868421, 0.36629257, 0.00633915, 0.05105334, 0.42458305,
       0.10001611, 0.14501777, 0.05399945, 0.23676595, 0.12034659,
       0.35720539, 0.06929339, 0.04867289, 0.38898978, 0.04295772,
       0.20864813, 0.02295152, 0.00910959, 0.70395726, 0.09370021,
       0.15618773, 0.0195958 , 0.04778903, 0.79669225, 0.07017403,
       0.83380061, 0.79950792, 0.2215552 , 0.02299087, 0.06304066,
       0.18589291, 0.00573243, 0.05516048, 0.86759204, 0.02548574,
       0.21893778, 0.14737961, 0.37895864, 0.09778561, 0.08328963,
       0.15278049, 0.01012357, 0.00627465, 0.76510048, 0.06883164,
       0.03448415, 0.13805096, 0.01701255, 0.01841686, 0.69192

# LOAD BEST MODEL FROM CBM PAPER

In [11]:

best_model = os.path.join(PROJECT_ROOT, 'models', 'best_model_1.pth')
model = torch.load(best_model, map_location=device, weights_only=False)
print("Best model loaded.")

if train_loader:
    with torch.no_grad():
        shuffled_labels = []

        # Iterate through all batches
        for batch in train_loader:
            _, _, image_labels, _ = batch
            # Append batch labels to our list
            shuffled_labels.append(image_labels)

        # Concatenate all batches into a single tensor
        shuffled_labels = torch.cat(shuffled_labels, dim=0)

        test_loss, test_acc, outputs = run_epoch_x_to_c(
            model, train_loader, attr_criterion, optimizer=None, n_concepts=N_TRIMMED_CONCEPTS, device=device,
            return_outputs='sigmoid', verbose=True
        )

# print(f"Shuffled labels shape: {shuffled_labels.shape}")
np.save(os.path.join(PROJECT_ROOT, 'output', 'Y_train_best_model_instance.npy'), shuffled_labels)
print(f'Best Model Summary   | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')

Best model loaded.


Best Model Summary   | Loss: 71.3566 | Acc: 82.323


In [12]:
output_array = get_outputs_as_array(outputs, N_TRIMMED_CONCEPTS)
np.save(os.path.join(PROJECT_ROOT, 'output', 'C_hat_sigmoid_train_best_model_instance.npy'), output_array)

In [ ]:
# best_model = os.path.join(PROJECT_ROOT, 'models', 'best_model_2.pth')
# model = torch.load(best_model, map_location=device, weights_only=False)
# print("Best model loaded.")

# if train_loader:
#     with torch.no_grad():
#         test_loss, test_acc = run_epoch_x_to_c(model, train_loader, attr_criterion, optimizer, n_concepts=N_TRIMMED_CONCEPTS, device=device)

# print(f'Best Model Train Summary   | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')

In [ ]:
# if test_loader:
#     with torch.no_grad():
#         test_loss, test_acc = run_epoch_x_to_c(model, test_loader, attr_criterion, optimizer, n_concepts=N_TRIMMED_CONCEPTS, device=device)

# print(f'Best Model Test Summary    | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')

In [ ]:
# best_model = os.path.join(PROJECT_ROOT, 'models', 'best_model_3.pth')
# model = torch.load(best_model, map_location=device, weights_only=False)
# print("Best model loaded.")

# if train_loader:
#     with torch.no_grad():
#         test_loss, test_acc = run_epoch_x_to_c(model, train_loader, attr_criterion, optimizer, n_concepts=N_TRIMMED_CONCEPTS, device=device)

# print(f'Best Model Train Summary   | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')

In [ ]:
# if test_loader:
#     with torch.no_grad():
#         test_loss, test_acc = run_epoch_x_to_c(model, test_loader, attr_criterion, optimizer, n_concepts=N_TRIMMED_CONCEPTS, device=device)

# print(f'Best Model Test Summary    | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')